In [1]:
import os
from os.path import join as pjoin

import cv2
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

#!pip install torchmetrics
import torchmetrics.classification as metrics

import torchvision
from torchvision import transforms

#!pip install -U segmentation-models-pytorch
import segmentation_models_pytorch as smp

from torch.utils.tensorboard import SummaryWriter

import albumentations as A

from tqdm import tqdm

#!pip install torchinfo
import torchinfo

import matplotlib.pyplot as plt

from torch.cuda import is_available
device = 'cuda' if is_available() else 'cpu'   # Меняем наш механизм на работу от гпу
print(device)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [2]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [3]:
import os
from os.path import join as pjoin
import contextlib


import cv2
import numpy as np


import matplotlib.pyplot as plt

def uniqufy_path(path):
    filename, extension = os.path.splitext(path)
    file_index = 1

    while os.path.exists(path):
        path = f"{filename}_{file_index}{extension}"
        file_index += 1
    
    return path

def create_image_plot(row_len : int = None, figsize = (16,6), **images):
    n_images = len(images)
    if row_len is None:
        row_len = n_images
    fig = plt.figure(figsize=figsize) 
    k = 0
    for idx, (name, image) in enumerate(images.items()):
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax = fig.add_subplot(n_images//row_len+1, row_len, idx+1)
        ax.set_title(name.title(), fontsize=16)
        if(k == 0):
          image = image / 8 + 0.5
          k = 1
        else:
          k = 0
        #print(np.amax(image))
        #print(np.amin(image))
        with open("$null", 'w') as dummy_f:
            with contextlib.redirect_stderr(dummy_f):
                ax.imshow(image)
    return fig

def save_imgs(path = None, name = "imgs", **images):
    if(path is None):
        raise AttributeError(f"You shoud write path")
    if not os.path.exists(path):
        os.makedirs(path)
    image_path = pjoin(path, f"{name}")
    fig = create_image_plot(**images)
    fig.savefig(image_path)
    fig.clear()
    plt.close(fig)

In [4]:
LAUNCH_NAME = "UNet_9"


STARTING_EPOCH = 0
LOAD_WEIGHTS = None #
LOAD_ADAM_STATE = None #
USE_MANUAL_TENSORBOARD_FOLDER = None # 

SAVED_MODEL_PATH = None 

EPOCHS = 15
LEARNING_RATE = 1E-5 # 0.0001 
WEIGHT_DECAY = 0 # 1E-7

BATCH_SIZE = 20 # 20

SAVE_METHOD = "TORCH" # "TORCH" / "ONNX"
WEIGHT_SAVER = "last" # "all" / "nothing" / "last"

CLASS_NAMES = ['other', 'road']
CLASS_RGB_VALUES = [[0,0,0], [255, 255, 255]]

NORMALIZE_MEAN_IMG =  [0.4295, 0.4325, 0.3961]       #[0.485, 0.456, 0.406]
NORMALIZE_DEVIATIONS_IMG =  [0.2267, 0.2192, 0.2240] #[0.229, 0.224, 0.225]
 
CROP_SIZE = (256, 256)

NUM_WORKERS = 4

# IMAGE_TRAIN_DATASET_PATH = "/usr/src/app/roads_dataset_cropped/tiff/train"
# MASK_TRAIN_DATASET_PATH = "/usr/src/app/roads_dataset_cropped/tiff/train_labels"
# IMAGE_TEST_DATASET_PATH = "/usr/src/app/roads_dataset_cropped/tiff/test"
# MASK_TEST_DATASET_PATH = "/usr/src/app/roads_dataset_cropped/tiff/test_labels"
# IMAGE_VALID_DATASET_PATH = "/usr/src/app/roads_dataset_cropped/tiff/val"
# MASK_VALID_DATASET_PATH = "/usr/src/app/roads_dataset_cropped/tiff/val_labels"

DATASET_DIR = '/usr/src/app/roads_dataset_cropped/tiff'
VALID_SET   = (pjoin(DATASET_DIR, "val"), pjoin(DATASET_DIR, "val_labels"))
TEST_SET   =  (pjoin(DATASET_DIR, "test"), pjoin(DATASET_DIR, "test_labels"))
TRAIN_SET   = (pjoin(DATASET_DIR, "train"), pjoin(DATASET_DIR, "train_labels"))

trained = False

In [5]:
TBpath = uniqufy_path(f"TB_cache/{LAUNCH_NAME}") if USE_MANUAL_TENSORBOARD_FOLDER is None else USE_MANUAL_TENSORBOARD_FOLDER
TBwriter = SummaryWriter(TBpath)

In [6]:
def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

prepare_to_network = A.Lambda(image=to_tensor, mask=to_tensor)

train_transform = A.Compose(
    [
        A.OneOf(
            [
                A.HorizontalFlip(p=1),
                A.VerticalFlip(p=1),
                A.RandomRotate90(p=1),
            ],
            p=0.75,
        ),
        A.Normalize(mean=NORMALIZE_MEAN_IMG, std=NORMALIZE_DEVIATIONS_IMG, always_apply=True)
    ]
)


valid_transform = A.Compose(
    [
        A.Normalize(mean=NORMALIZE_MEAN_IMG, std=NORMALIZE_DEVIATIONS_IMG, always_apply=True),
    ]
)

In [7]:
def one_hot_encode(label, label_values):
    semantic_map = []
    for colour in label_values:
        equality = np.equal(label, colour)
        class_map = np.all(equality, axis = -1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1)
    return semantic_map

def reverse_one_hot(image):
    x = np.argmax(image, axis = -1)
    return x

def colour_code_segmentation(image, label_values):
    colour_codes = np.array(label_values)
    # print(np.amax(image))
    # print(np.amin(image))
    # print(image)
    x = colour_codes[(image).astype(int)]
    # print(np.amax(image))
    # print(np.amin(image))
    # print(image)
    return x

In [8]:
class RoadsDataset(Dataset):
    def __init__(self, values_dir, labels_dir, class_rgb_values=None, transform=None, readyToNetwork=None):
        self.values_dir = values_dir
        self.labels_dir = labels_dir
        self.class_rgb_values = class_rgb_values
        self.images = [pjoin(self.values_dir, filename) for filename in sorted(os.listdir(self.values_dir))]
        self.labels = [pjoin(self.labels_dir, filename) for filename in sorted(os.listdir(self.labels_dir))]
        self.transform = transform
        self.readyToNetwork = readyToNetwork

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = self.images[index]
        label_path = self.labels[index]

        image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        label = cv2.cvtColor(cv2.imread(label_path), cv2.COLOR_BGR2RGB)
        label = one_hot_encode(label, self.class_rgb_values).astype('float')

        if self.transform:
            sample = self.transform(image=image, mask=label)
            image, label = sample['image'], sample['mask']
        if self.readyToNetwork:
            sample = self.readyToNetwork(image=image, mask=label)
            image, label = sample['image'], sample['mask']
        return image, label

In [9]:
sample_dataset = RoadsDataset(*TEST_SET,
                       class_rgb_values=CLASS_RGB_VALUES, transform=valid_transform)

for i in range(10):
    image, mask = sample_dataset[np.random.randint(0, len(sample_dataset))]
    TBwriter.add_figure(f'train samples', create_image_plot(origin=image, true=colour_code_segmentation(reverse_one_hot(mask), CLASS_RGB_VALUES)), global_step=i)
del(sample_dataset)

In [10]:
import segmentation_models_pytorch as smp

ENCODER = 'resnet50'
CLASSES = CLASS_NAMES
ACTIVATION = nn.ReLU

model = smp.Unet(
    encoder_name=ENCODER, 
    classes=len(CLASSES),
    activation=ACTIVATION,
)



In [12]:
# model = UNet()
model = model.to(device)
print(model_sum := torchinfo.summary(model, depth=10, input_size=(BATCH_SIZE, 3, *CROP_SIZE), row_settings=["var_names"], verbose=0, col_names=[
      "input_size", "output_size", "num_params", "params_percent", "kernel_size", "mult_adds", "trainable"]))

Layer (type (var_name))                            Input Shape               Output Shape              Param #                   Param %                   Kernel Shape              Mult-Adds                 Trainable
Unet (Unet)                                        [20, 3, 256, 256]         [20, 2, 256, 256]         --                             --                   --                        --                        True
├─ResNetEncoder (encoder)                          [20, 3, 256, 256]         [20, 3, 256, 256]         --                             --                   --                        --                        True
│    └─Conv2d (conv1)                              [20, 3, 256, 256]         [20, 64, 128, 128]        9,408                       0.03%                   [7, 7]                    3,082,813,440             True
│    └─BatchNorm2d (bn1)                           [20, 64, 128, 128]        [20, 64, 128, 128]        128                         0.00%           

In [13]:
train_dataset = RoadsDataset(*TRAIN_SET,
                       class_rgb_values=CLASS_RGB_VALUES, transform=train_transform, readyToNetwork=prepare_to_network)
valid_dataset = RoadsDataset(*VALID_SET,
                       class_rgb_values=CLASS_RGB_VALUES, transform=valid_transform, readyToNetwork=prepare_to_network)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,  
)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE//4,
    num_workers=NUM_WORKERS,
)

In [14]:
# model = UNet().to(device)

In [15]:
loss = smp.losses.DiceLoss(mode='binary')

optimizer = torch.optim.Adam(model.parameters(),
                              lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [16]:
def train_step(net, criterion, optimizer, dataloader, epoch: int = None):
    net.train()
    running_loss = 0.
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        output = net(images)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss

    with torch.no_grad():
        train_loss = running_loss / len(dataloader)
    return train_loss.item()

s = nn.Softmax()
def valid_step(net, criterion, dataloader, epoch: int = None):
    net.eval()
    running_loss = 0.
    IoU = metrics.BinaryJaccardIndex()
    IoU.to(device)

    with torch.no_grad():
        for step, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            output = net(images)

            t=s(output)
            IoU(output, labels)
            loss = criterion(output, labels)
            running_loss += loss

            save_imgs(pjoin(TBpath, f"valid_samples/samples_{epoch}"), name=f"img_{step}",
                origin=images[0].cpu().numpy().transpose(2, 1, 0),
                true=colour_code_segmentation(reverse_one_hot(
                    labels[0].cpu().numpy().transpose(2, 1, 0)), CLASS_RGB_VALUES),
                pred=colour_code_segmentation(reverse_one_hot(
                    output[0].cpu().numpy().transpose(2, 1, 0)), CLASS_RGB_VALUES))

        TBwriter.add_figure('valid_sample', create_image_plot(
                origin=images[0].cpu().numpy().transpose(2, 1, 0),
                true=colour_code_segmentation(reverse_one_hot(
                    labels[0].cpu().numpy().transpose(2, 1, 0)), CLASS_RGB_VALUES),
                pred=colour_code_segmentation(reverse_one_hot(
                    output[0].cpu().numpy().transpose(2, 1, 0)), CLASS_RGB_VALUES)),
                  epoch)

        valid_loss = running_loss / len(valid_dataloader)

        return valid_loss.item(), IoU.compute().item()

In [18]:
epoch = STARTING_EPOCH

best_loss = 10000
trained = True

pbar = tqdm(range(EPOCHS))
pbar.update(epoch)

while(epoch < EPOCHS):
    train_loss = train_step(model, loss, optimizer, train_dataloader, epoch)
    valid_loss, iou_score = valid_step(model, loss, valid_dataloader, epoch)
    scheduler.step(valid_loss)

    if WEIGHT_SAVER != "nothing" and valid_loss < best_loss and epoch > 1:
        best_loss = valid_loss

        print(f"[{epoch}] Saved weights with IoU: {iou_score:.2f} | loss: {valid_loss:.4f}")
    
#         if WEIGHT_SAVER == "last":
#             weights_path = f"{TBpath}/weights_last.pth"
#             model_path =   f"{TBpath}/model_last.onnx"
#             optimizer_path = f"{TBpath}/optimizer_last.pth"

#         if "TORCH" in SAVE_METHOD:
#             torch.save(model.state_dict(), weights_path)


    TBwriter.add_scalar('valid loss', valid_loss, epoch)
    TBwriter.add_scalar('train loss', train_loss, epoch)
    
    TBwriter.add_scalar('IoU', iou_score, epoch)

    epoch += 1
    pbar.update()
    pbar.set_description(
        f'IoU: {iou_score:.2f}  | train/valid loss: {train_loss:.4f}/{valid_loss:.4f}')

  0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipykernel_883373/3358363367.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  t=s(output)
IoU: 0.89  | train/valid loss: 0.2462/0.2430:  20%|██        | 3/15 [17:06<1:08:18, 341.52s/it]

[2] Saved weights with IoU: 0.89 | loss: 0.2430


IoU: 0.92  | train/valid loss: 0.2286/0.2275:  27%|██▋       | 4/15 [23:58<1:07:40, 369.17s/it]

[3] Saved weights with IoU: 0.92 | loss: 0.2275


IoU: 0.92  | train/valid loss: 0.2196/0.2207:  33%|███▎      | 5/15 [30:02<1:01:15, 367.60s/it]

...

[12] Saved weights with IoU: 0.93 | loss: 0.2128


IoU: 0.93  | train/valid loss: 0.2089/0.2127:  93%|█████████▎| 14/15 [1:21:24<05:43, 343.25s/it]

[13] Saved weights with IoU: 0.93 | loss: 0.2127


IoU: 0.93  | train/valid loss: 0.2088/0.2126: 100%|██████████| 15/15 [1:27:05<00:00, 342.56s/it]

[14] Saved weights with IoU: 0.93 | loss: 0.2126


In [19]:
test_transform = A.Compose(
    [
        A.Normalize(mean=NORMALIZE_MEAN_IMG, std=NORMALIZE_DEVIATIONS_IMG, always_apply=True),
    ]
)

test_dataset = RoadsDataset(*TEST_SET,
       class_rgb_values=CLASS_RGB_VALUES, transform=valid_transform, readyToNetwork=prepare_to_network)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=36,
    num_workers=NUM_WORKERS,
)
if not trained:
    print(f"Используется не обученная модель, происходит загрузка модели из {SAVED_MODEL_PATH}")
    model = None
    if "TORCH" in SAVE_METHOD and model is None:
        print(f"Попытка импорта модели из pth файла")
        model = UNet(3,2,bilinear=True)
        model.state_dict(torch.load(f=SAVED_MODEL_PATH))

    model.to(device)

In [20]:
def test_step(model, loader):
    classes = CLASS_NAMES

    iou = metrics.JaccardIndex(task="binary", num_classes=2).to(device)

    with torch.no_grad():
        model.eval()
        model.to(device)
        for id, (images, labels) in enumerate(loader):
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            TBwriter.add_figure('test_sample', create_image_plot(
                origin=images[0].cpu().numpy().transpose(2, 1, 0),
                true=colour_code_segmentation(reverse_one_hot(
                    labels[0].cpu().numpy().transpose(2, 1, 0)), CLASS_RGB_VALUES),
                pred=colour_code_segmentation(reverse_one_hot(
                    output[0].cpu().numpy().transpose(2, 1, 0)), CLASS_RGB_VALUES)),
                  id)
            iou.update(output, labels)
    return iou.compute()

In [21]:
iou = test_step(model, test_dataloader)
print(f"IoU: {iou}")
TBwriter.close()

IoU: 0.9571388959884644


In [22]:
import numpy as np

# Определите фильтр для выполнения билинейной интерполяции
def bilinear_filter(size):
    factor = (size + 1) // 2
    if size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:size, :size]
    filter = (1 - abs(og[0] - center) / factor) * (1 - abs(og[1] - center) / factor)
    return filter

filter_size = 4
filter = bilinear_filter(filter_size) 

import torch
import torch.nn as nn

in_channels = 3  # количество входных каналов (цветовых каналов)
out_channels = 3  # количество выходных каналов
stride = 2  # шаг (stride)

conv_transpose = nn.ConvTranspose2d(in_channels, out_channels, filter_size, stride=stride, bias=False)
conv_transpose.weight.data = torch.Tensor(np.tile(filter, (in_channels, out_channels, 1, 1)))